## Perform intensity normalization for brain MRI images;

Notes: 
- References:https://github.com/jcreinhold/intensity-normalization.
- We only implement Z-score and FCM-based normalization method here.
- Assert the brain MRI images are all skull-stripped!

In [ ]:
import os
import shutil
import ants
from mySettings import get_intensity_normalization_setting_dict

import sys
sys.path.append("../")
from utils.myUtils import traversalDir_FirstDir, mkdir, get_filenames

In [ ]:
def resave_no_normalized_images(image_folder, modality_list, save_basepath):
    """
    Resave the images into different modality folders. 
    """  
    patient_list=traversalDir_FirstDir(image_folder)

    for modality in modality_list:
        save_modality_image_folder=os.path.join(save_basepath, "no_normalization", modality)
        mkdir(save_modality_image_folder)
        for patient_id in patient_list:
            image_basename=patient_id+"_"+modality+".nii.gz"
            patient_modality_image_path=os.path.join(image_folder, patient_id, image_basename)
            resave_patient_modality_image_path=os.path.join(save_modality_image_folder, image_basename)
            shutil.copy(patient_modality_image_path, resave_patient_modality_image_path)
    
    print("Finish resaving the original images into different modality folders!")
    
def check_image_spacing(image_folder, modality_list, image_spacing=(1, 1, 1)):
    """
    Check whether the images are resampled to 1mmx1mmx1mm or not.
    """
    image_resampled=True
    patient_list=traversalDir_FirstDir(image_folder)

    for modality in modality_list:
        for patient_id in patient_list:
            image_basename=patient_id+"_"+modality+".nii.gz"
            patient_modality_image_path=os.path.join(image_folder, patient_id, image_basename)
            
            img = ants.image_read(patient_modality_image_path)
            if img.spacing != image_spacing:
                print("patient_id={}, modality={}, img.spacing={}.".format(patient_id, modality, img.spacing))
                image_resampled=False
                
    print("All the images are resampled? Answer: {}.".format(image_resampled))
    return image_resampled
     
    
def perform_normalization(normalized_image_basepath, modality_list, normalization_method):
    '''
    perform intensity normalization for brain MRI images, with individual timepoint-based normalization method;
    
    References: https://github.com/jcreinhold/intensity-normalization.
    '''
    print("\n -normalization_method={},  -modality_list={}.".format(normalization_method, modality_list))
    print("\n -normalized_image_basepath={}.\n".format(normalized_image_basepath))
    
    for modality in modality_list:
        # folder to save the normalized images for this modality.
        save_normalized_image_folder=os.path.join(normalized_image_basepath, normalization_method, modality)
        mkdir(save_normalized_image_folder)
        
        # unnormalized-image path list for this modality.
        modality_image_folder=os.path.join(normalized_image_basepath, "no_normalization", modality)
        modality_imagePath_list=get_filenames(modality_image_folder)
        
        # create folder to save white matter masks from FCM normalization method.
        if normalization_method=="fcm":
            save_t1_wm_folder=os.path.join(normalized_image_basepath, normalization_method, "t1_wm")
            mkdir(save_t1_wm_folder)
        
        # normalize the image one by one.
        for image_path in modality_imagePath_list:
            image_base_name=os.path.basename(image_path)
            save_normalized_image_path=os.path.join(save_normalized_image_folder, image_base_name)
            
            mo=modality if modality in ["t2", "flair", "md", "pd", "t1"] else "other"
            #--------- fcm normalization ------------
            if normalization_method=="fcm":
                if modality=="t1":
                    ! fcm-normalize $image_path -o $save_normalized_image_path -mo $mo -tt wm
                    wm_image_basename=image_base_name.replace(".nii.gz", "_wm_membership.nii.gz")
                    wm_image_path=os.path.join(save_normalized_image_folder, wm_image_basename)
                    resave_wm_image_path=os.path.join(save_t1_wm_folder, wm_image_basename)
                    shutil.move(wm_image_path, resave_wm_image_path)
                else:
                    t1_wm_image_basename=image_base_name.split("_")[0]+"_t1_wm_membership.nii.gz"
                    wm_image_path=os.path.join(save_t1_wm_folder, t1_wm_image_basename)
                    ! fcm-normalize $image_path -tm $wm_image_path -o $save_normalized_image_path -mo $mo

            elif normalization_method=="zscore":
                ! zscore-normalize $image_path -o $save_normalized_image_path -mo $mo
            
            else:
                raise Error("Undefined normalization method!")
                
    print("Finish normalizing the images by {} method!".format(normalization_method))
    

def resave_normalized_images(normalized_image_basepath, modality_list, normalization_method):
    """
    Resave the normalized images to make sure that the images from one patient are saved in a separate folder.
    """
    # Base path for resaving the images.
    resave_normalized_image_basepath=os.path.join(normalized_image_basepath, normalization_method, normalization_method+"_normalizedImages")
    
    # Get an example of the patient image path list.
    example_modality_image_folder=os.path.join(normalized_image_basepath, normalization_method, modality_list[0])
    example_patient_list=get_filenames(example_modality_image_folder)
    print("There are {} patients!".format(len(example_patient_list)))
    
    # Resave the image for each patient.
    for patient_image_path in example_patient_list:
        patient_id=os.path.basename(patient_image_path).split("_")[0]
        save_normalized_images_for_patient=os.path.join(resave_normalized_image_basepath, patient_id)
        mkdir(save_normalized_images_for_patient)
        for modality in modality_list:
            image_basename=patient_id+"_"+modality+".nii.gz"
            orig_image_path=os.path.join(normalized_image_basepath, normalization_method, modality, image_basename)
            resaved_image_path=os.path.join(save_normalized_images_for_patient, image_basename)
            shutil.copy(orig_image_path, resaved_image_path)
        
    print("Finish resaving the normalized images!") 
    
def plot_image_histograms(normalized_image_basepath, modality_list, normalization_method):
    """
    Plot image histograms to compare the images before and after normalization;
    """
    for modality in modality_list:
        image_folder=os.path.join(normalized_image_basepath, normalization_method, modality)
        save_fig_path=os.path.join(normalized_image_basepath, normalization_method+"_"+modality+"_imghist.jpeg")
        ! plot-histograms $image_folder -o $save_fig_path
        
    print("Finish ploting the image histograms for the normalized images!")

### Main

In [ ]:
intensity_normalization_setting_dict=get_intensity_normalization_setting_dict()
for setting_name, intensity_normalization_setting in intensity_normalization_setting_dict.items():
    normalization_method=intensity_normalization_setting["normalization_method"]
    image_folder=intensity_normalization_setting["image_folder"]
    modality_list=intensity_normalization_setting["modality_list"]
    normalized_image_basepath=intensity_normalization_setting["normalized_image_basepath"]
    
    print("\n ====== Perform {} intensity normalization for {} data... =======".format(normalization_method, setting_name))
    
    #check the image spacing to ensure all the images are resampled to (1,1,1).
    check_image_spacing(image_folder, modality_list)
    
    #resave the non-normalized the images into different modality folders.
    resave_no_normalized_images(image_folder, modality_list, normalized_image_basepath)
    
    #perform intensity normalization;
    perform_normalization(normalized_image_basepath, modality_list, normalization_method)
    
    #resave the normalized images to make sure that the images from one patient are saved in a separate folder.
    resave_normalized_images(normalized_image_basepath, modality_list, normalization_method)
    
    #plot image histograms to compare the images before and after normalization;
    plot_image_histograms(normalized_image_basepath, modality_list, normalization_method)